### Introduction
This notebook aims to demostrate how to implement Apriori algorithm to discover potential 'rules' in a data set.

Given a list of sets, the algorithm will find the 'rules' and present the 'rules' in the format of ｛A1, A2, …｝->｛B1, B2, …｝


The 'rules' are discovered based on these two criteria: P(A1,A2,…,B1,B2,…) >= min_support and P(B1, B2, …|A1, A2, …) >= min_confident

min_support and min_confident are user specified values

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import itertools
import re

In [2]:
class apriori:
    def __init__(self, min_support, min_confidence):
        self.sets_count = dict()
        self.min_support_sets = None
        self.min_support = min_support
        self.min_confidence = min_confidence
    
    def get_set_count(self, data, set_candidate):
        '''
        Return number of occurrence of set_candidate in the data

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.
        
        set_candidate: Set
            A set of items to be counted

        Returns
        ----------
        Int:
            Number of set_candidate in data
        '''  
        item_key = tuple(set_candidate)
        if item_key not in self.sets_count:
            item_count = 0
            for record in data:
                if set_candidate.issubset(record):
                    item_count += 1
            self.sets_count[item_key] = item_count
        return self.sets_count[item_key]
    
    def gen_first_sets(self, data):
        '''
        Return lists of candidate sets. Each set has one item

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.

        Returns
        ----------
        List:
            A list of sets with length one
        '''  
        res = set()
        for record in data:
            res.update(record)
        res = [{i} for i in res]
        return res
    
    def gen_CTable(self, data, subsets):
        '''
        Return support_value for each item in subsets which is greater than min_support

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.
        
        subsets: List
            A list of items that need to be counted. Each item should a set

        Returns
        ----------
        Dict: 
            A dictionary mapping items in subsets and corresponding support value.
            format: {tuple : float}
        '''     
        CTable = defaultdict(lambda: 0)
        for item in subsets:
            CTable[tuple(item)] = self.get_set_count(data, item)
        l = len(data)
        for k in list(CTable.keys()):
            if CTable[k]/l < self.min_support:
                CTable.pop(k)
            else:
                CTable[k] /= l
        return CTable
    
    def is_super_set(self, candidate_set, recorded_sets):
        '''
        If any subset of candidate_set is in recorded_sets, return True.
        e.g. candidate_set = {1,2,3}   recorded_sets = {(5,7), (9,), (1,3)}   Return: True 
        Because {1,3} is one of subset of {1,2,3} and it is in recorded_sets

        Parameters
        -----------
        candidate_set: set
            The set to be tested
        
        recorded_sets: Set
            Set of tuples

        Returns
        ----------
        Bool:
            Whether candidate_set is super set of any set in recorded_sets
        '''
        l = len(candidate_set)
        for k in range(1, l):
            sub_candidates = list(itertools.combinations(candidate_set, k))
            for sub_candidate in sub_candidates:
                if sub_candidate in recorded_sets:
                    return True
        return False
 
    
    def gen_k_sets(self, prev_candidates, k):
        '''
        Return lists of candidate sets. Each set has k items

        Parameters
        -----------
        prev_candidates: set
            Set of tuples of satisfying min_support
        
        k: int
            Specify the length of sets generated

        Returns
        ----------
        List:
            A list of sets with length k
        '''
        c_prev = [set(i) for i in prev_candidates]  # convert to list of sets
        c_next = list(itertools.combinations(c_prev, 2)) # list of tuples of 2 sets
        c_next = {tuple(set1.union(set2)) for set1, set2 in c_next if len(tuple(set1.union(set2))) == k}  # convert to set of tuples with length k
        # all subsets with length k-1 of candidate should fulfill min_support
        remove_list = []
        for candidate in c_next:
            sub_candidates = itertools.combinations(candidate, k-1)
            for sub_candidate in sub_candidates:
                if sub_candidate not in prev_candidates:
                    remove_list.append(candidate)
                    break
        for candidate in remove_list:
            c_next.remove(candidate)
        c_next = [set(i) for i in c_next]
        return c_next
    
    def get_min_support_sets(self, data):
        '''
        Return subsets with support greater than min_support

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.

        Returns
        ----------
        Dict:
            A dictionary mapping subsets and corresponding support values
        '''
        # Generate count table for set with length 1
        c_dict = dict()    # c_dict[k] store the CTable of length k
        subsets = self.gen_first_sets(data)
        ck = self.gen_CTable(data, subsets)
        c_dict[1] = ck
        k=2
        while len(c_dict[k-1]) > 1:
            subsets = self.gen_k_sets(set(c_dict[k-1].keys()), k)
            ck = self.gen_CTable(data, subsets)
            c_dict[k] = ck
            k+=1
        self.min_support_sets = c_dict
        return self.min_support_sets
    
    def get_confidence(self, data, left_set, right_set):
        '''
        Return confidence value for the rule 'left_set -> right_set'

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.
        
        left_set: set
            A set of items on left hand side of rule
        
        right_set: set
            A set of items on right hand side of rule

        Returns
        ----------
        Float:
            Confidence value of the rule
        '''
        left_count = self.get_set_count(data, left_set)
        combined_count = self.get_set_count(data, left_set.union(right_set))
        return combined_count/left_count
        
        
    def find_rules(self, data):
        '''
        Return rules with confidence value greater than min_confidence

        Parameters
        -----------
        data: List
            A list of records. Each records should be a set.

        Returns
        ----------
        Dict:
            A dictionary mapping rules and corresponding confidence values
        '''
        hist = defaultdict(lambda: set())
        res = dict()
        l = len(data)
        if self.min_support_sets is None:
            self.get_min_support_sets(data)
        for k in self.min_support_sets.keys():
            if k == 1:
                continue
            for subset in self.min_support_sets[k].keys():
                for num_left in range(1,k):   # num_left is number of items on left hand side of the rule 
                    num_right = k - num_left
                    left_candidates = list(itertools.combinations(subset, num_left))
                    right_candidates = list(itertools.combinations(subset, num_right))
                    # for each combination of rules
                    for left_candidate in left_candidates:
                        for right_candidate in right_candidates:
                            if set(left_candidate).issubset(set(right_candidate)) or set(right_candidate).issubset(set(left_candidate)):
                                continue
                            right_candidate = tuple(set(right_candidate) - set(left_candidate))
                            # Given left_candidate, if right_candidate is superset of a 'past right_candidate' with confidence value smaller than required value,
                            # then its confidence value must be smaller than required value
                            if self.is_super_set(set(right_candidate), hist[left_candidate]):
                                hist[left_candidate].add(right_candidate)
                                continue
                            else:
                                confidence_val = self.get_confidence(data, set(left_candidate), set(right_candidate))
                                support = self.get_set_count(data, set(left_candidate).union(set(right_candidate)))/l
                                if confidence_val >= self.min_confidence:
                                    str_output = '{} -> {}'.format(left_candidate, right_candidate)
                                    if str_output not in res:
                                        print('Rule: {}  Support: {}  Confidence: {}'.format(str_output, support, confidence_val))
                                        res[str_output] = confidence_val
                                else:
                                    hist[left_candidate].add(right_candidate)
        return res
                        
                    
                    
                
            

### Demostration

In [3]:
t = [{1,3,4},{2,3,5},{1,2,3,5},{2,5}]  # input data should be a list of sets
a = apriori(min_support=0.5, min_confidence=0.5)
d = a.get_min_support_sets(t)  # return sets that fulfill min_support requirement
for k, v in d.items():
    print('length of bag: ',k)
    for bag, support_val in list(v.items()):
        print('bag = {},    support = {}'.format(bag,support_val))

length of bag:  1
bag = (1,),    support = 0.5
bag = (2,),    support = 0.75
bag = (3,),    support = 0.75
bag = (5,),    support = 0.75
length of bag:  2
bag = (1, 3),    support = 0.5
bag = (2, 3),    support = 0.5
bag = (2, 5),    support = 0.75
bag = (3, 5),    support = 0.5
length of bag:  3
bag = (2, 3, 5),    support = 0.5


In [4]:
rules = a.find_rules(t)

Rule: (1,) -> (3,)  Support: 0.5  Confidence: 1.0
Rule: (3,) -> (1,)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (2,) -> (3,)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (3,) -> (2,)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (2,) -> (5,)  Support: 0.75  Confidence: 1.0
Rule: (5,) -> (2,)  Support: 0.75  Confidence: 1.0
Rule: (3,) -> (5,)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (5,) -> (3,)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (2,) -> (3, 5)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (3,) -> (2, 5)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (5,) -> (2, 3)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (2, 3) -> (5,)  Support: 0.5  Confidence: 1.0
Rule: (2, 5) -> (3,)  Support: 0.5  Confidence: 0.6666666666666666
Rule: (3, 5) -> (2,)  Support: 0.5  Confidence: 1.0


###### Explanation

Consider Rule: (2, 5) -> (3,)  Support: 0.5  Confidence: 0.6666666666666666

Support_value is 0.5 because there are 2 sets containing {2,5,3} out of 4 sets in data.

Confidence_value is 0.667 because there are 2 sets containing {2,5,3} out of 3 sets containing {2,5}

Only rules with support_value > 0.5 and confidence_value > 0.5 will be printed